In [1]:
# Copyright 2021 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

import requests
from PIL import Image
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

2022-11-16 10:54:32.694293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 10:54:32.737282: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


The following GPU devices are available: /device:GPU:0


2022-11-16 10:54:33.657363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 10:54:33.713498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 10:54:33.733313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 10:54:33.733771: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one

In [3]:
original_image_cache = {}

def preprocess_image(image):
  image = np.array(image)
  # reshape into shape [batch_size, height, width, num_channels]
  img_reshaped = tf.reshape(image, [1, image.shape[0], image.shape[1], image.shape[2]])
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  image = tf.image.convert_image_dtype(img_reshaped, tf.float32)
  return image

def load_image_from_url(img_url):
  """Returns an image with shape [1, height, width, num_channels]."""
  user_agent = {'User-agent': 'Colab Sample (https://tensorflow.org)'}
  response = requests.get(img_url, headers=user_agent)
  image = Image.open(BytesIO(response.content))
  image = preprocess_image(image)
  return image

def load_image(image_url, image_size=256, dynamic_size=False, max_dynamic_size=512):
  """Loads and preprocesses images."""
  # Cache image file locally.
  if image_url in original_image_cache:
    img = original_image_cache[image_url]
  elif image_url.startswith('https://'):
    img = load_image_from_url(image_url)
  else:
    fd = tf.io.gfile.GFile(image_url, 'rb')
    img = preprocess_image(Image.open(fd))
  original_image_cache[image_url] = img
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img_raw = img
  if tf.reduce_max(img) > 1.0:
    img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  if not dynamic_size:
    img = tf.image.resize_with_pad(img, image_size, image_size)
  elif img.shape[1] > max_dynamic_size or img.shape[2] > max_dynamic_size:
    img = tf.image.resize_with_pad(img, max_dynamic_size, max_dynamic_size)
  return img, img_raw

def show_image(image, title=''):
  image_size = image.shape[1]
  w = (image_size * 6) // 320
  plt.figure(figsize=(w, w))
  plt.imshow(image[0], aspect='equal')
  plt.axis('off')
  plt.title(title)
  plt.show()

In [4]:
labels_file = "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt"

#download labels and creates a maps
downloaded_file = tf.keras.utils.get_file("labels.txt", origin=labels_file)

classes = []

with open(downloaded_file) as f:
  labels = f.readlines()
  classes = [l.strip() for l in labels]
    

In [5]:
model_list = [
    "googlenet[incepv1](2014)",
    "inception_v3(2015)", 
    "inception_resnet_v2(2016)", 
    "mobilenet(2017)",
    "mobilenet-v2(2018)",
    "mobilenet-v3(2019)",
    "bit-s-resnet101x3-1k(2020)",
    "bit-m-resnet101x3-1k(2020)",
    "efficientnet-v2-l-1k(2021)",
    "mlp-mixer-l-1k(2021)"
]
#ay caramba, el problema testo here
#model_list = ["mlp-mixer-l-1k(2021)"]
#images_list = ["tiger"]
images_list = [
    "tiger", 
    "bus", 
    "car", 
    "cat", 
    "dog", 
    "apple", 
    "banana", 
    "turtle", 
    "flamingo", 
    "piano", 
    "honeycomb", 
    "teapot",
    "beaver",
    "cannon",
    "forklift",
    "pillow",
    "radio",
    "snorkel",
    "stove",
    "tractor",
    "yurt",
    "pretzel"
]
modelcount = []
highpasses = []
passes = []
lowpasses = []
rawscore = []
totalmodelcount = 0
for _ in range(len(model_list)):
    modelcount.append(0)
    rawscore.append(0)
    highpasses.append(0)
    passes.append(0)
    lowpasses.append(0)
for j in model_list:
    toggle = 0
    model_name = j
    model_handle_map = {
      "googlenet[incepv1](2014)": "https://tfhub.dev/google/imagenet/inception_v1/classification/5",
      "inception_v3(2015)": "https://tfhub.dev/google/imagenet/inception_v3/classification/4",
      "inception_resnet_v2(2016)": "https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/4",
      "mobilenet(2017)": "https://tfhub.dev/google/imagenet/mobilenet_v1_100_224/classification/5",
      "mobilenet-v2(2018)": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5",
      "mobilenet-v3(2019)": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5",
      "bit-s-resnet101x3-1k(2020)": "https://tfhub.dev/google/bit/s-r101x3/ilsvrc2012_classification/1",
      "bit-m-resnet101x3-1k(2020)": "https://tfhub.dev/google/bit/m-r101x3/ilsvrc2012_classification/1",
      "efficientnet-v2-l-1k(2021)": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_l/classification/2",
      "mlp-mixer-l-1k(2021)": "https://tfhub.dev/sayakpaul/mixer_l16_i1k_classification/1"
    }

    model_image_size_map = {
      "googlenet[incepv1](2014)": 224,
      "inception_v3(2015)": 299,
      "inception_resnet_v2(2016)": 299,
      "mobilenet(2017)": 224,
      "mobilenet-v2(2018)": 224,
      "mobilenet-v3(2019)": 224,
      "bit-s-resnet101x3-1k(2020)": 224,
      "bit-m-resnet101x3-1k(2020)": 224,
      "efficientnet-v2-l-1k(2021)": 480,
      "mlp-mixer-l-1k(2021)": 224
    }

    model_handle = model_handle_map[model_name]

    #print(f"Selected model: {model_name} : {model_handle}")
    for l in images_list:

        image_name = l 
        image_size = 224
        dynamic_size = False
        max_dynamic_size = 512
        images_for_test_map = {
            "tiger": "https://upload.wikimedia.org/wikipedia/commons/b/b0/Bengal_tiger_%28Panthera_tigris_tigris%29_female_3_crop.jpg",
            "bus": "https://upload.wikimedia.org/wikipedia/commons/6/63/LT_471_%28LTZ_1471%29_Arriva_London_New_Routemaster_%2819522859218%29.jpg",
            "car": "https://upload.wikimedia.org/wikipedia/commons/4/49/2013-2016_Toyota_Corolla_%28ZRE172R%29_SX_sedan_%282018-09-17%29_01.jpg",
            "cat": "https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/CatVibrissaeFullFace.JPG/1200px-CatVibrissaeFullFace.JPG",
            "dog": "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a7/Bakarwal.jpg/1200px-Bakarwal.jpg",
            "apple": "https://upload.wikimedia.org/wikipedia/commons/1/15/Red_Apple.jpg",
            "banana": "https://upload.wikimedia.org/wikipedia/commons/1/1c/Bananas_white_background.jpg",
            "turtle": "https://upload.wikimedia.org/wikipedia/commons/8/80/Turtle_golfina_escobilla_oaxaca_mexico_claudio_giovenzana_2010.jpg",
            "flamingo": "https://upload.wikimedia.org/wikipedia/commons/b/b8/James_Flamingos_MC.jpg",
            "piano": "https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Steinway_%26_Sons_concert_grand_piano%2C_model_D-274%2C_manufactured_at_Steinway%27s_factory_in_Hamburg%2C_Germany.png/1200px-Steinway_%26_Sons_concert_grand_piano%2C_model_D-274%2C_manufactured_at_Steinway%27s_factory_in_Hamburg%2C_Germany.png",
            "honeycomb": "https://upload.wikimedia.org/wikipedia/commons/f/f7/Honey_comb.jpg",
            "teapot": "https://upload.wikimedia.org/wikipedia/commons/4/44/Black_tea_pot_cropped.jpg",
            "beaver": "https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Beaver_pho34.jpg/640px-Beaver_pho34.jpg",
            "cannon": "https://upload.wikimedia.org/wikipedia/commons/4/4f/Canon_obusier_de_campagne_de_12_modele_1853.jpg",
            "forklift": "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/1956_Toyota_Model_LA_Forklift_01.jpg/1200px-1956_Toyota_Model_LA_Forklift_01.jpg",
            "pillow": "https://upload.wikimedia.org/wikipedia/commons/a/ad/Pillows_on_a_hotel_bed.jpg",
            "radio": "https://upload.wikimedia.org/wikipedia/commons/thumb/f/f3/Radio.jpg/717px-Radio.jpg",
            "snorkel": "https://upload.wikimedia.org/wikipedia/commons/8/8c/Snorkel.jpg",
            "stove": "https://upload.wikimedia.org/wikipedia/commons/9/99/Gas_stove.jpg",
            "tractor": "https://upload.wikimedia.org/wikipedia/commons/3/3f/Nuffield_tractor%2C_Cophill_Farm_vintage_rally_2012.jpg",
            "yurt": "https://upload.wikimedia.org/wikipedia/commons/thumb/4/43/Kyrgyzsk%C3%A1_jurta%2C_Song-k%C3%B6l.jpg/640px-Kyrgyzsk%C3%A1_jurta%2C_Song-k%C3%B6l.jpg",
            "pretzel": "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRmwqOgdCHrdf02pO5wI2TeFKkU2kwTgNkuAg&usqp=CAU",
        }

        img_url = images_for_test_map[image_name]
        image, original_image = load_image(img_url, image_size, dynamic_size, max_dynamic_size)
        #show_image(image, 'Scaled image')

        if model_name in model_image_size_map:
            image_size = model_image_size_map[model_name]
            image_size = tf.cast(image_size, tf.float32)
            dynamic_size = False
            #print(f"Images will be converted to {image_size}x{image_size}")
        else:
            dynamic_size = True
            #print(f"Images will be capped to a max size of {max_dynamic_size}x{max_dynamic_size}")



        #class
        classifier = hub.load(model_handle)

        input_shape = image.shape
        warmup_input = tf.random.uniform(input_shape, 0, 1.0)
        try:
            if(toggle == 0):
                %time warmup_logits = classifier(warmup_input).numpy()
                toggle = 1
            else:
                warmup_logits = classifier(warmup_input).numpy()
            
            
            # Run model on image
            #%time probabilities = tf.nn.softmax(classifier(image)).numpy()
            probabilities = tf.nn.softmax(classifier(image)).numpy()
            top_1 = tf.argsort(probabilities, axis=-1, direction="DESCENDING")[0][:1].numpy()
            np_classes = np.array(classes)

            # Some models include an additional 'background' class in the predictions, so
            # we must account for this when reading the class labels.
            includes_background_class = probabilities.shape[1] == 1001

            for i, item in enumerate(top_1):
                class_index = item if includes_background_class else item + 1
                #line = f'({i+1}) {class_index:4} - {classes[class_index]}: {probabilities[0][top_1][i]}'
                #print(line)
                try:
                    line = classes[class_index]
                    if(image_name in line):
                        if(probabilities[0][top_1][i] < 0.5):
                            passed = "Low Pass"
                            lowpasses[totalmodelcount] += 1
                        elif(probabilities[0][top_1][i] < 0.75):
                            passed = "Pass"
                            passes[totalmodelcount] += 1
                        else:
                            passed = "High Pass"
                            highpasses[totalmodelcount] += 1
                        rawscore[totalmodelcount] += probabilities[0][top_1][i]
                    else:
                        passed = "Fail"
                    print(line + ": " + f'{probabilities[0][top_1][i]*100:3.2f}' + ", " + passed + " - " + model_name)
                #show_image(image, '')
                except IndexError:
                    passed = "Fail"
                    print("none" + ": " + f'{0:3.2f}' + ", " + passed + " - " + model_name)
        except Exception as e:
            print(model_name + " failed to run.")
            print(e)
    totalmodelcount+=1
    print()
print("Ready.")


2022-11-16 10:54:36.602082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 10:54:36.602604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 10:54:36.603039: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 10:54:36.603578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-16 10:54:36.603985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

CPU times: user 1.33 s, sys: 429 ms, total: 1.75 s
Wall time: 1.8 s
tiger: 76.20, High Pass - googlenet[incepv1](2014)
school bus: 49.93, Low Pass - googlenet[incepv1](2014)
sports car: 22.63, Low Pass - googlenet[incepv1](2014)
Egyptian cat: 33.46, Low Pass - googlenet[incepv1](2014)
dingo: 19.60, Fail - googlenet[incepv1](2014)
Granny Smith: 85.97, Fail - googlenet[incepv1](2014)
banana: 88.74, High Pass - googlenet[incepv1](2014)
leatherback turtle: 80.89, High Pass - googlenet[incepv1](2014)
flamingo: 71.69, Pass - googlenet[incepv1](2014)
grand piano: 99.92, High Pass - googlenet[incepv1](2014)
honeycomb: 88.47, High Pass - googlenet[incepv1](2014)
teapot: 93.97, High Pass - googlenet[incepv1](2014)
beaver: 96.93, High Pass - googlenet[incepv1](2014)
cannon: 82.54, High Pass - googlenet[incepv1](2014)
forklift: 94.99, High Pass - googlenet[incepv1](2014)
studio couch: 19.05, Fail - googlenet[incepv1](2014)
radio: 77.68, High Pass - googlenet[incepv1](2014)
snorkel: 17.33, Low Pass

2022-11-16 11:12:16.208994: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x5597d8ddbc80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-11-16 11:12:16.209023: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti, Compute Capability 8.6
2022-11-16 11:12:16.385413: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-11-16 11:12:19.229554: I tensorflow/compiler/jit/xla_compilation_cache.cc:476] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


CPU times: user 2.54 s, sys: 57 µs, total: 2.54 s
Wall time: 3.31 s
tiger: 78.43, High Pass - mlp-mixer-l-1k(2021)
minibus: 14.23, Low Pass - mlp-mixer-l-1k(2021)
beach wagon: 66.22, Fail - mlp-mixer-l-1k(2021)
Egyptian cat: 40.45, Low Pass - mlp-mixer-l-1k(2021)
malinois: 26.37, Fail - mlp-mixer-l-1k(2021)
orange: 10.03, Fail - mlp-mixer-l-1k(2021)
banana: 99.98, High Pass - mlp-mixer-l-1k(2021)
leatherback turtle: 99.80, High Pass - mlp-mixer-l-1k(2021)
flamingo: 90.23, High Pass - mlp-mixer-l-1k(2021)
grand piano: 77.05, High Pass - mlp-mixer-l-1k(2021)
honeycomb: 79.02, High Pass - mlp-mixer-l-1k(2021)
teapot: 53.15, Pass - mlp-mixer-l-1k(2021)
beaver: 68.45, Pass - mlp-mixer-l-1k(2021)
cannon: 98.72, High Pass - mlp-mixer-l-1k(2021)
forklift: 80.22, High Pass - mlp-mixer-l-1k(2021)
quilt: 48.65, Fail - mlp-mixer-l-1k(2021)
radio: 51.30, Pass - mlp-mixer-l-1k(2021)
snorkel: 88.00, High Pass - mlp-mixer-l-1k(2021)
stove: 68.24, Pass - mlp-mixer-l-1k(2021)
thresher: 77.67, Fail - mlp

In [6]:
for k in range(totalmodelcount):
    print("75+%: " + f'{highpasses[k]}' + ", 50%-75%: " + f'{passes[k]}' + ", <50%: " + f'{lowpasses[k]}' +
          " Average acc: " + f'{rawscore[k]/len(images_list)*100:3.2f}' + 
          "% - " + model_list[k])

75+%: 12, 50%-75%: 3, <50%: 4 Average acc: 64.29% - googlenet[incepv1](2014)
75+%: 18, 50%-75%: 0, <50%: 0 Average acc: 79.73% - inception_v3(2015)
75+%: 16, 50%-75%: 0, <50%: 0 Average acc: 71.95% - inception_resnet_v2(2016)
75+%: 12, 50%-75%: 2, <50%: 4 Average acc: 66.04% - mobilenet(2017)
75+%: 10, 50%-75%: 4, <50%: 3 Average acc: 57.10% - mobilenet-v2(2018)
75+%: 10, 50%-75%: 5, <50%: 3 Average acc: 61.71% - mobilenet-v3(2019)
75+%: 14, 50%-75%: 5, <50%: 0 Average acc: 75.52% - bit-s-resnet101x3-1k(2020)
75+%: 14, 50%-75%: 3, <50%: 0 Average acc: 71.84% - bit-m-resnet101x3-1k(2020)
75+%: 0, 50%-75%: 0, <50%: 17 Average acc: 18.11% - efficientnet-v2-l-1k(2021)
75+%: 11, 50%-75%: 4, <50%: 2 Average acc: 57.28% - mlp-mixer-l-1k(2021)
